## Packages

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

from deepctr.models import DeepFM
from deepctr.inputs import SparseFeat, VarLenSparseFeat,get_feature_names
import pandas_profiling
from sklearn.model_selection import train_test_split

##  Process Language and Locations            

In [70]:
Campaign=pd.read_csv("Campaign.csv")

/home/yxy/anaconda3/envs/py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (4,6,7,8,9,13,14,15,16,17,18,21,22,24,26,27,28,29,30,31,32,33,34,35,42,44,46,48,54,56,57,62,65,66) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [72]:
print(len(pd.unique(Campaign["languageCode"])))
print(len(pd.unique(Campaign["locationName"])))

47
1025


In [103]:
CampaignLoc=Campaign[Campaign["criterionType"]=="LOCATION"][["campaignId","locationName"]].groupby(['campaignId'])   
CampaignLoc=CampaignLoc["locationName"].apply(list)
print(CampaignLoc.head())
CampaignLANG.to_csv("LANGUAGE_processed.csv")
CampaignLoc.to_csv("Location_processed.csv")

campaignId
3807257         [Hong Kong]
20572439    [United States]
20858009    [United States]
21659909        [Singapore]
35073532    [United States]
Name: locationName, dtype: object


In [98]:
CampaignLANG=pd.read_csv("LANGUAGE_processed.csv")
CampaignLoc=pd.read_csv("Location_processed.csv")

Campaign = pd.merge(CampaignLoc,
                 CampaignLANG,
                 how='inner', on="campaignId"
                 )

print(Campaign.head())
Campaign.to_csv("Campaign_processed.csv")


   campaignId       locationName     languageCode
0     3807257      ['Hong Kong']  ['en', 'zh_TW']
1    20572439  ['United States']           ['en']
2    20858009  ['United States']           ['en']
3    21659909      ['Singapore']           ['en']
4    35073532  ['United States']           ['en']


## Filter data with impression > threshold

In [40]:
data=pd.read_csv("./dac/Adwords/AW_PlacementPerformanceReport_201912201603.csv",chunksize=1000000)
i=0
for placement in data:
    i+=1
    print("ith iter",i)
    c=placement['Ctr'].apply(lambda x: 1 if x > 0 else 0)
    # Remove records with less than 100 Impressions
    grouped = placement[(placement["Impressions"] >= 20)]
    grouped=grouped[['BaseAdGroupId','Criteria','CampaignId','ActiveViewCpm','ActiveViewCtr','ActiveViewImpressions','ActiveViewMeasurability','ActiveViewMeasurableCost','ActiveViewMeasurableImpressions',
                     'ActiveViewViewability','AverageCpc','Impressions','Ctr']]
    if i==1:
        grouped.to_csv("./dac/Adwords/impression20.csv",mode='a') 
    else:
        grouped.to_csv("./dac/Adwords/impression20.csv",mode='a',header=False) 

ith iter 1
ith iter 2
ith iter 3
ith iter 4
ith iter 5
ith iter 6
ith iter 7
ith iter 8
ith iter 9
ith iter 10
ith iter 11
ith iter 12
ith iter 13
ith iter 14
ith iter 15
ith iter 16
ith iter 17
ith iter 18
ith iter 19
ith iter 20
ith iter 21


## Split Before Process

In [3]:
from sklearn.model_selection import train_test_split
data=pd.read_csv("./dac/Adwords/impression20.csv")


In [15]:
train=pd.read_csv("./dac/Adwords/Train2020.csv")
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4962248 entries, 0 to 4962247
Data columns (total 15 columns):
Unnamed: 0                         int64
Unnamed: 0.1                       int64
BaseAdGroupId                      int64
Criteria                           object
CampaignId                         int64
ActiveViewCpm                      float64
ActiveViewCtr                      float64
ActiveViewImpressions              int64
ActiveViewMeasurability            float64
ActiveViewMeasurableCost           float64
ActiveViewMeasurableImpressions    int64
ActiveViewViewability              float64
AverageCpc                         float64
Impressions                        int64
Ctr                                float64
dtypes: float64(7), int64(7), object(1)
memory usage: 567.9+ MB


In [14]:
test=pd.read_csv("./dac/Adwords/Test2020.csv")
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1240562 entries, 0 to 1240561
Data columns (total 15 columns):
Unnamed: 0                         1240562 non-null int64
Unnamed: 0.1                       1240562 non-null int64
BaseAdGroupId                      1240562 non-null int64
Criteria                           1240562 non-null object
CampaignId                         1240562 non-null int64
ActiveViewCpm                      1240562 non-null float64
ActiveViewCtr                      1240562 non-null float64
ActiveViewImpressions              1240562 non-null int64
ActiveViewMeasurability            1240562 non-null float64
ActiveViewMeasurableCost           1240562 non-null float64
ActiveViewMeasurableImpressions    1240562 non-null int64
ActiveViewViewability              1240562 non-null float64
AverageCpc                         1240562 non-null float64
Impressions                        1240562 non-null int64
Ctr                                1240562 non-null float64
dt

## Process the adGroup

In [29]:
def preprocessing(filename,n_Placement,n_Adgroup,mode,trainResult,testResult):
    placement = pd.read_csv(filename)
    print("start process")
    
    grouped=placement.groupby(["BaseAdGroupId","Criteria"]).mean()
    # Filter data with higher 
    grouped = grouped.groupby(['BaseAdGroupId'])
    grouped = grouped.filter(lambda x: len(x) > n_Placement)
    grouped = grouped.groupby(['Criteria'])
    grouped = grouped.filter(lambda x: len(x) > n_Adgroup)
    train,test=train_test_split(grouped,test_size=0.2,random_state=1024)
    train.to_csv("./dac/Adwords/grouped_train.csv")
    test.to_csv("./dac/Adwords/grouped_test.csv")
    train=pd.read_csv("./dac/Adwords/grouped_train.csv")
    test=pd.read_csv("./dac/Adwords/grouped_test.csv")
    test.info()

    print("Saving statistics")
    forAvg=train[['BaseAdGroupId','Criteria','ActiveViewCpm','ActiveViewCtr','ActiveViewImpressions','ActiveViewMeasurability','ActiveViewMeasurableCost','ActiveViewMeasurableImpressions',
                     'ActiveViewViewability','AverageCpc','Impressions','Ctr']]
    
    adGroupAvg=forAvg.groupby('BaseAdGroupId').mean()

    adGroupAvg.columns = ['adActiveViewCpm','adActiveViewCtr','adActiveViewImpressions','adActiveViewMeasurability','adActiveViewMeasurableCost',
                          'adActiveViewMeasurableImpressions','adActiveViewViewability','adAverageCpc','adImpressions','adCtr']

    placementAvg=forAvg.groupby("Criteria").mean()
    placementAvg.columns=["BaseAdGroupId",'plaActiveViewCpm','plaActiveViewCtr','plaActiveViewImpressions','plaActiveViewMeasurability','plaActiveViewMeasurableCost',
                          'plaActiveViewMeasurableImpressions','plaActiveViewViewability','plaAverageCpc','plaImpressions','plaCtr']
    placementAvg = placementAvg.drop(columns="BaseAdGroupId")
    adGroupAvg.to_csv("./dac/Adwords/adGroupAvg.csv")
    placementAvg.to_csv("./dac/Adwords/placementAvg.csv")
    print("Filter Criteria: {}".format(grouped.shape[0]))
    c=grouped['Ctr'].apply(lambda x: 1 if x>0 else 0)
    print(c.value_counts())
    print("Start parse")   
#     for dataset in [train,test]:
#         data = []
#         y = []
#         exec("grouped={}".format(dataset))
#         grouped.info()
#         print("------------------------------Finally----------------------------")
#         for row in grouped.iterrows():
#             adGroupId, placement,CampaignId, Ctr = row[0][0], row[0][1], row[1]['CampaignId'],row[1]['Ctr']
#             data.append({"BaseAdGroupId": str(adGroupId), "Criteria": str(placement),"campaignId":CampaignId,"placementType":"mobileapp" if placement.find("mobileapp::")>-1 else "web"})
#             y.append(float(Ctr))
#         result=pd.DataFrame.from_dict(data)
#         result.insert(0,'target',np.array(y))
    train=train[['BaseAdGroupId','Criteria']]
    placementAvg=pd.read_csv("./dac/Adwords/placementAvg.csv")
    print("merge Criteria")
    train= pd.merge(train,
                 placementAvg,
                 how='left', on="Criteria")
    print("Merge Placement matrics")
    adGroupAvg=pd.read_csv("./dac/Adwords/adGroupAvg.csv")
    adGroupAvg["BaseAdGroupId"] = adGroupAvg["BaseAdGroupId"].astype(str)
    train["BaseAdGroupId"]=train["BaseAdGroupId"].astype(str)
    train= pd.merge(train,
                 adGroupAvg,
                 how='left', on="BaseAdGroupId"
                 )
    print(train.info())
    train.to_csv(trainResult)
    
    test=test[['BaseAdGroupId','Criteria']]
    placementAvg=pd.read_csv("./dac/Adwords/placementAvg.csv")
    print("merge Criteria")
    test= pd.merge(test,
                 placementAvg,
                 how='left', on="Criteria")
    print("Merge Placement matrics")
    adGroupAvg=pd.read_csv("./dac/Adwords/adGroupAvg.csv")
    adGroupAvg["BaseAdGroupId"] = adGroupAvg["BaseAdGroupId"].astype(str)
    test["BaseAdGroupId"]=test["BaseAdGroupId"].astype(str)
    test= pd.merge(test,
                 adGroupAvg,
                 how='left', on="BaseAdGroupId"
                 )
    print(test.info())
    test.to_csv(testResult)

In [30]:
preprocessing("./dac/Adwords/impression20.csv",20,20,"Train","./dac/Adwords/train2020Processed.csv","./dac/Adwords/test2020Processed.csv")

start process
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172522 entries, 0 to 172521
Data columns (total 14 columns):
BaseAdGroupId                      172522 non-null int64
Criteria                           172522 non-null object
Unnamed: 0                         172522 non-null float64
CampaignId                         172522 non-null float64
ActiveViewCpm                      172522 non-null float64
ActiveViewCtr                      172522 non-null float64
ActiveViewImpressions              172522 non-null float64
ActiveViewMeasurability            172522 non-null float64
ActiveViewMeasurableCost           172522 non-null float64
ActiveViewMeasurableImpressions    172522 non-null float64
ActiveViewViewability              172522 non-null float64
AverageCpc                         172522 non-null float64
Impressions                        172522 non-null float64
Ctr                                172522 non-null float64
dtypes: float64(12), int64(1), object(1)
memory usag

In [18]:
data=pd.read_csv("./dac//Adwords/impression20.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6202810 entries, 0 to 6202809
Data columns (total 14 columns):
Unnamed: 0                         int64
BaseAdGroupId                      int64
Criteria                           object
CampaignId                         int64
ActiveViewCpm                      float64
ActiveViewCtr                      float64
ActiveViewImpressions              int64
ActiveViewMeasurability            float64
ActiveViewMeasurableCost           float64
ActiveViewMeasurableImpressions    int64
ActiveViewViewability              float64
AverageCpc                         float64
Impressions                        int64
Ctr                                float64
dtypes: float64(7), int64(6), object(1)
memory usage: 662.5+ MB


In [53]:
data=pd.read_csv("./dac//Adwords/adGroup_processed2020.csv")
print(len(pd.unique(data['Criteria'])))
print(len(pd.unique(data['BaseAdGroupId'])))

7775
4403


In [43]:
preprocessing("./dac/Adwords/impression20.csv",20,20,20,100,100,"./dac/Adwords/adGroup_processed2020.csv")

start process
Filter Criteria: 862608  
0    573525
1    289083
Name: Ctr, dtype: int64
Start parse
------------------------------Finally----------------------------
merge Criteria
  BaseAdGroupId     Criteria   campaignId placementType  target  \
0    4391795246   143mov.com  111013286.0           web     0.0   
1    4391795246   151mov.com  111013286.0           web     0.0   
2    4391795246   153mov.com  111013286.0           web     0.0   
3    4391795246   157mov.com  111013286.0           web     0.0   
4    4391795246  17track.net  111013286.0           web     0.0   

   plaActiveViewCpm  plaActiveViewCtr  plaActiveViewImpressions  \
0          9.815985          0.370241                257.283039   
1         11.549335          0.335896                185.008934   
2          6.674319          0.392401                 49.235725   
3          6.109229          0.190103                168.325549   
4          6.564894          0.674315                106.153415   

   plaActiveV

In [ ]:
import datetime
#long running
#do something other 
placement = pd.read_csv("./dac/Adwords/impression50.csv")
print("start profiling")

starttime = datetime.datetime.now()
pandas_profiling.ProfileReport(placement)

endtime = datetime.datetime.now()
print (endtime - starttime)

In [3]:
placement = pd.read_csv("./dac/Adwords/impression50.csv")
print(placement.describe())

         Unnamed: 0
count  2.934303e+06
mean   1.034290e+07
std    6.189671e+06
min    1.900000e+01
25%    4.772822e+06
50%    1.024509e+07
75%    1.582623e+07
max    2.098998e+07


In [42]:
df=pd.read_csv("./dac/Adwords/impression20.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6202810 entries, 0 to 6202809
Data columns (total 14 columns):
Unnamed: 0                         int64
BaseAdGroupId                      int64
Criteria                           object
CampaignId                         int64
ActiveViewCpm                      float64
ActiveViewCtr                      float64
ActiveViewImpressions              int64
ActiveViewMeasurability            float64
ActiveViewMeasurableCost           float64
ActiveViewMeasurableImpressions    int64
ActiveViewViewability              float64
AverageCpc                         float64
Impressions                        int64
Ctr                                float64
dtypes: float64(7), int64(6), object(1)
memory usage: 662.5+ MB


In [4]:
def joinAdGroupCampain(adGroupFile,CampaignFile,resultFile):
    adGroup=pd.read_csv(adGroupFile)
    campaign=pd.read_csv(CampaignFile)
    processed= pd.merge(adGroup,
                 campaign,
                 how='inner', on="campaignId"
                 )
  
    processed.drop(columns="Unnamed: 0_x")

    print(processed.head())
    processed.to_csv(resultFile)

In [47]:
joinAdGroupCampain("./dac/Adwords/adGroup_processed20.csv","Campaign_processed.csv")

   Unnamed: 0_x  BaseAdGroupId       Criteria   campaignId placementType  \
0             0     4391795246     143mov.com  111013286.0           web   
1             1     4391795246     157mov.com  111013286.0           web   
2             2     4391795246  23minutos.com  111013286.0           web   
3             3     4391795246     881903.com  111013286.0           web   
4             4     4391795246   aastocks.com  111013286.0           web   

     target  plaActiveViewCpm  plaActiveViewCtr  plaActiveViewImpressions  \
0  0.000000          9.097298          0.331782                452.898172   
1  0.000000          5.566029          0.220095                300.758651   
2  1.300000         15.893750          0.685250                 41.387500   
3  0.000000          9.405371          0.527551                 90.701422   
4  0.010123          1.171664          0.026114               1541.453040   

   plaActiveViewMeasurability  ...  adActiveViewMeasurability  \
0              

In [56]:
joinAdGroupCampain("./dac/Adwords/adGroup_processed2020.csv","Campaign_processed.csv","./dac/Adwords/AllFeatures2020.csv")

   Unnamed: 0_x  BaseAdGroupId     Criteria   campaignId placementType  \
0             0     4391795246   143mov.com  111013286.0           web   
1             1     4391795246   151mov.com  111013286.0           web   
2             2     4391795246   153mov.com  111013286.0           web   
3             3     4391795246   157mov.com  111013286.0           web   
4             4     4391795246  17track.net  111013286.0           web   

   target  plaActiveViewCpm  plaActiveViewCtr  plaActiveViewImpressions  \
0     0.0          9.815985          0.370241                257.283039   
1     0.0         11.549335          0.335896                185.008934   
2     0.0          6.674319          0.392401                 49.235725   
3     0.0          6.109229          0.190103                168.325549   
4     0.0          6.564894          0.674315                106.153415   

   plaActiveViewMeasurability  ...  adActiveViewMeasurability  \
0                   90.020929  ...     